In [31]:
from pathlib import Path

import numpy as np
import pandas as pd

In [32]:
data=pd.read_csv("train.csv",index_col="tripid")
test=pd.read_csv("test.csv",index_col="tripid")

In [33]:
data.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            16974 non-null  float64
 1   duration                   16974 non-null  float64
 2   meter_waiting              16974 non-null  float64
 3   meter_waiting_fare         16974 non-null  float64
 4   meter_waiting_till_pickup  16974 non-null  float64
 5   pickup_time                17176 non-null  object 
 6   drop_time                  17176 non-null  object 
 7   pick_lat                   17176 non-null  float64
 8   pick_lon                   17176 non-null  float64
 9   drop_lat                   17176 non-null  float64
 10  drop_lon                   17176 non-null  float64
 11  fare                       17039 non-null  float64
 12  label                      17176 non-null  object 
dtypes: float64(10), object(3)
memory u

In [35]:
features_df=data.drop('label',axis=1)
labels_df=data['label'].replace(['correct','incorrect'],[1,0])

In [36]:
features_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39


In [37]:
labels_df.head(25)

tripid
189123628    1
189125358    1
189125719    1
189127273    1
189128020    1
189129552    1
189132829    1
189135103    1
189139296    1
189138671    1
189135814    0
189140969    0
189144528    1
189147041    0
189145904    1
189146174    1
189150541    1
189153229    1
189151585    1
189157464    1
189157607    0
189157476    1
189157559    1
189159951    1
189158726    1
Name: label, dtype: int64

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

import seaborn as sns

from catboost import CatBoostClassifier

RANDOM_SEED = 6

In [39]:
numeric_cols = features_df.columns[features_df.dtypes != "object"].values
not_numeric_cols =features_df.columns[features_df.dtypes == "object"].values

In [40]:
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='median'))
])


categorical_features = not_numeric_cols
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols)
    ],
    remainder = "drop"
)

In [41]:
model = CatBoostClassifier()

In [42]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", model),
])

In [43]:
full_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler()),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  array(['additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare'], dtype=object))])),
                ('estimators',
                 <catboost.core.CatBoostClassifier object at 0x00000240555AC1C8>)])

In [44]:
X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=RANDOM_SEED
)

In [45]:
y_train.head()

tripid
205667210    0
193277385    1
201668211    1
204263911    0
206929715    1
Name: label, dtype: int64

In [46]:
full_pipeline.fit(X_train, y_train)

Learning rate set to 0.029239
0:	learn: 0.6646699	total: 12.9ms	remaining: 12.9s
1:	learn: 0.6347958	total: 25.4ms	remaining: 12.7s
2:	learn: 0.6067671	total: 39.3ms	remaining: 13.1s
3:	learn: 0.5841312	total: 54.9ms	remaining: 13.7s
4:	learn: 0.5594673	total: 72.6ms	remaining: 14.5s
5:	learn: 0.5373968	total: 90.6ms	remaining: 15s
6:	learn: 0.5195950	total: 110ms	remaining: 15.6s
7:	learn: 0.4997500	total: 131ms	remaining: 16.3s
8:	learn: 0.4840085	total: 154ms	remaining: 16.9s
9:	learn: 0.4689200	total: 179ms	remaining: 17.7s
10:	learn: 0.4539370	total: 214ms	remaining: 19.3s
11:	learn: 0.4416936	total: 245ms	remaining: 20.1s
12:	learn: 0.4269535	total: 271ms	remaining: 20.6s
13:	learn: 0.4162883	total: 303ms	remaining: 21.3s
14:	learn: 0.4058921	total: 328ms	remaining: 21.5s
15:	learn: 0.3955547	total: 353ms	remaining: 21.7s
16:	learn: 0.3860103	total: 383ms	remaining: 22.2s
17:	learn: 0.3768814	total: 414ms	remaining: 22.6s
18:	learn: 0.3656783	total: 442ms	remaining: 22.8s
19:	lea

167:	learn: 0.1787868	total: 4.36s	remaining: 21.6s
168:	learn: 0.1784584	total: 4.39s	remaining: 21.6s
169:	learn: 0.1782458	total: 4.42s	remaining: 21.6s
170:	learn: 0.1779000	total: 4.45s	remaining: 21.6s
171:	learn: 0.1774934	total: 4.47s	remaining: 21.5s
172:	learn: 0.1773453	total: 4.5s	remaining: 21.5s
173:	learn: 0.1769515	total: 4.52s	remaining: 21.5s
174:	learn: 0.1767127	total: 4.55s	remaining: 21.4s
175:	learn: 0.1764129	total: 4.57s	remaining: 21.4s
176:	learn: 0.1758629	total: 4.59s	remaining: 21.4s
177:	learn: 0.1756946	total: 4.62s	remaining: 21.3s
178:	learn: 0.1754315	total: 4.64s	remaining: 21.3s
179:	learn: 0.1751461	total: 4.66s	remaining: 21.2s
180:	learn: 0.1746306	total: 4.68s	remaining: 21.2s
181:	learn: 0.1744273	total: 4.71s	remaining: 21.2s
182:	learn: 0.1742264	total: 4.74s	remaining: 21.1s
183:	learn: 0.1740370	total: 4.76s	remaining: 21.1s
184:	learn: 0.1738599	total: 4.79s	remaining: 21.1s
185:	learn: 0.1736140	total: 4.81s	remaining: 21.1s
186:	learn: 0

329:	learn: 0.1489144	total: 8.34s	remaining: 16.9s
330:	learn: 0.1487921	total: 8.37s	remaining: 16.9s
331:	learn: 0.1487005	total: 8.4s	remaining: 16.9s
332:	learn: 0.1485473	total: 8.42s	remaining: 16.9s
333:	learn: 0.1483660	total: 8.45s	remaining: 16.9s
334:	learn: 0.1482774	total: 8.48s	remaining: 16.8s
335:	learn: 0.1480779	total: 8.5s	remaining: 16.8s
336:	learn: 0.1479690	total: 8.53s	remaining: 16.8s
337:	learn: 0.1478837	total: 8.55s	remaining: 16.8s
338:	learn: 0.1477942	total: 8.58s	remaining: 16.7s
339:	learn: 0.1476629	total: 8.61s	remaining: 16.7s
340:	learn: 0.1475970	total: 8.64s	remaining: 16.7s
341:	learn: 0.1474286	total: 8.67s	remaining: 16.7s
342:	learn: 0.1473448	total: 8.71s	remaining: 16.7s
343:	learn: 0.1471805	total: 8.74s	remaining: 16.7s
344:	learn: 0.1470819	total: 8.77s	remaining: 16.6s
345:	learn: 0.1469919	total: 8.79s	remaining: 16.6s
346:	learn: 0.1468834	total: 8.81s	remaining: 16.6s
347:	learn: 0.1467571	total: 8.84s	remaining: 16.6s
348:	learn: 0.

491:	learn: 0.1315056	total: 12.4s	remaining: 12.8s
492:	learn: 0.1313571	total: 12.4s	remaining: 12.8s
493:	learn: 0.1312487	total: 12.4s	remaining: 12.7s
494:	learn: 0.1310072	total: 12.5s	remaining: 12.7s
495:	learn: 0.1308649	total: 12.5s	remaining: 12.7s
496:	learn: 0.1307996	total: 12.5s	remaining: 12.7s
497:	learn: 0.1306367	total: 12.5s	remaining: 12.6s
498:	learn: 0.1305019	total: 12.6s	remaining: 12.6s
499:	learn: 0.1304360	total: 12.6s	remaining: 12.6s
500:	learn: 0.1303913	total: 12.6s	remaining: 12.6s
501:	learn: 0.1302836	total: 12.6s	remaining: 12.5s
502:	learn: 0.1302435	total: 12.7s	remaining: 12.5s
503:	learn: 0.1301561	total: 12.7s	remaining: 12.5s
504:	learn: 0.1300954	total: 12.7s	remaining: 12.4s
505:	learn: 0.1299650	total: 12.7s	remaining: 12.4s
506:	learn: 0.1298818	total: 12.8s	remaining: 12.4s
507:	learn: 0.1298295	total: 12.8s	remaining: 12.4s
508:	learn: 0.1296902	total: 12.8s	remaining: 12.4s
509:	learn: 0.1295430	total: 12.8s	remaining: 12.3s
510:	learn: 

655:	learn: 0.1173169	total: 16.7s	remaining: 8.76s
656:	learn: 0.1172343	total: 16.7s	remaining: 8.74s
657:	learn: 0.1171460	total: 16.8s	remaining: 8.71s
658:	learn: 0.1170844	total: 16.8s	remaining: 8.69s
659:	learn: 0.1170375	total: 16.8s	remaining: 8.66s
660:	learn: 0.1169786	total: 16.8s	remaining: 8.64s
661:	learn: 0.1168986	total: 16.9s	remaining: 8.61s
662:	learn: 0.1167875	total: 16.9s	remaining: 8.59s
663:	learn: 0.1167428	total: 16.9s	remaining: 8.56s
664:	learn: 0.1167011	total: 16.9s	remaining: 8.54s
665:	learn: 0.1166627	total: 17s	remaining: 8.52s
666:	learn: 0.1165796	total: 17s	remaining: 8.49s
667:	learn: 0.1163869	total: 17s	remaining: 8.46s
668:	learn: 0.1162660	total: 17.1s	remaining: 8.44s
669:	learn: 0.1161686	total: 17.1s	remaining: 8.41s
670:	learn: 0.1160280	total: 17.1s	remaining: 8.39s
671:	learn: 0.1159011	total: 17.1s	remaining: 8.37s
672:	learn: 0.1158301	total: 17.2s	remaining: 8.35s
673:	learn: 0.1158000	total: 17.2s	remaining: 8.32s
674:	learn: 0.1157

817:	learn: 0.1062069	total: 21.2s	remaining: 4.72s
818:	learn: 0.1060964	total: 21.2s	remaining: 4.7s
819:	learn: 0.1060267	total: 21.3s	remaining: 4.67s
820:	learn: 0.1058969	total: 21.3s	remaining: 4.65s
821:	learn: 0.1058512	total: 21.3s	remaining: 4.62s
822:	learn: 0.1058099	total: 21.4s	remaining: 4.59s
823:	learn: 0.1057306	total: 21.4s	remaining: 4.57s
824:	learn: 0.1056750	total: 21.4s	remaining: 4.54s
825:	learn: 0.1056250	total: 21.5s	remaining: 4.52s
826:	learn: 0.1055214	total: 21.5s	remaining: 4.49s
827:	learn: 0.1054506	total: 21.5s	remaining: 4.47s
828:	learn: 0.1053907	total: 21.5s	remaining: 4.44s
829:	learn: 0.1053449	total: 21.6s	remaining: 4.42s
830:	learn: 0.1053056	total: 21.6s	remaining: 4.39s
831:	learn: 0.1052593	total: 21.6s	remaining: 4.37s
832:	learn: 0.1051580	total: 21.6s	remaining: 4.34s
833:	learn: 0.1051173	total: 21.7s	remaining: 4.31s
834:	learn: 0.1050815	total: 21.7s	remaining: 4.29s
835:	learn: 0.1050406	total: 21.7s	remaining: 4.26s
836:	learn: 0

978:	learn: 0.0967752	total: 25.4s	remaining: 545ms
979:	learn: 0.0967285	total: 25.4s	remaining: 519ms
980:	learn: 0.0966868	total: 25.5s	remaining: 493ms
981:	learn: 0.0966685	total: 25.5s	remaining: 467ms
982:	learn: 0.0966108	total: 25.5s	remaining: 441ms
983:	learn: 0.0965837	total: 25.5s	remaining: 415ms
984:	learn: 0.0965273	total: 25.5s	remaining: 389ms
985:	learn: 0.0964668	total: 25.6s	remaining: 363ms
986:	learn: 0.0964329	total: 25.6s	remaining: 337ms
987:	learn: 0.0963461	total: 25.6s	remaining: 311ms
988:	learn: 0.0963116	total: 25.6s	remaining: 285ms
989:	learn: 0.0962580	total: 25.7s	remaining: 259ms
990:	learn: 0.0961958	total: 25.7s	remaining: 233ms
991:	learn: 0.0961161	total: 25.7s	remaining: 207ms
992:	learn: 0.0960699	total: 25.7s	remaining: 181ms
993:	learn: 0.0960390	total: 25.8s	remaining: 155ms
994:	learn: 0.0960019	total: 25.8s	remaining: 130ms
995:	learn: 0.0959438	total: 25.8s	remaining: 104ms
996:	learn: 0.0958844	total: 25.8s	remaining: 77.7ms
997:	learn:

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler()),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  array(['additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare'], dtype=object))])),
                ('estimators',
                 <catboost.core.CatBoostClassifier object at 0x00000240555AC1C8>)])

In [47]:
preds = full_pipeline.predict_proba(X_eval)
preds

array([[0.02335653, 0.97664347],
       [0.02785552, 0.97214448],
       [0.02826621, 0.97173379],
       ...,
       [0.07233694, 0.92766306],
       [0.34466297, 0.65533703],
       [0.02708659, 0.97291341]])

In [48]:
y_preds = pd.DataFrame(
    {
        "label": preds[:, 1],
    },
    index = y_eval.index
)

In [49]:
roc_auc_score(y_eval, y_preds)

0.9193716594968061

In [50]:
full_pipeline.fit(features_df, labels_df)

Learning rate set to 0.034693
0:	learn: 0.6596663	total: 20.2ms	remaining: 20.1s
1:	learn: 0.6248031	total: 41.9ms	remaining: 20.9s
2:	learn: 0.5927717	total: 63.7ms	remaining: 21.2s
3:	learn: 0.5664193	total: 86.1ms	remaining: 21.4s
4:	learn: 0.5406512	total: 114ms	remaining: 22.7s
5:	learn: 0.5154182	total: 141ms	remaining: 23.4s
6:	learn: 0.4958854	total: 169ms	remaining: 24s
7:	learn: 0.4732702	total: 195ms	remaining: 24.2s
8:	learn: 0.4570583	total: 228ms	remaining: 25.1s
9:	learn: 0.4377078	total: 254ms	remaining: 25.2s
10:	learn: 0.4227764	total: 281ms	remaining: 25.3s
11:	learn: 0.4104257	total: 310ms	remaining: 25.5s
12:	learn: 0.3951172	total: 341ms	remaining: 25.9s
13:	learn: 0.3837881	total: 365ms	remaining: 25.7s
14:	learn: 0.3735471	total: 394ms	remaining: 25.9s
15:	learn: 0.3630301	total: 423ms	remaining: 26s
16:	learn: 0.3545622	total: 445ms	remaining: 25.7s
17:	learn: 0.3471640	total: 470ms	remaining: 25.6s
18:	learn: 0.3359087	total: 492ms	remaining: 25.4s
19:	learn: 

164:	learn: 0.1736334	total: 5.14s	remaining: 26s
165:	learn: 0.1732879	total: 5.17s	remaining: 26s
166:	learn: 0.1729872	total: 5.2s	remaining: 25.9s
167:	learn: 0.1728276	total: 5.23s	remaining: 25.9s
168:	learn: 0.1726239	total: 5.25s	remaining: 25.8s
169:	learn: 0.1722804	total: 5.28s	remaining: 25.8s
170:	learn: 0.1721561	total: 5.3s	remaining: 25.7s
171:	learn: 0.1718484	total: 5.33s	remaining: 25.7s
172:	learn: 0.1716094	total: 5.36s	remaining: 25.6s
173:	learn: 0.1713384	total: 5.39s	remaining: 25.6s
174:	learn: 0.1710370	total: 5.41s	remaining: 25.5s
175:	learn: 0.1708683	total: 5.44s	remaining: 25.5s
176:	learn: 0.1706623	total: 5.46s	remaining: 25.4s
177:	learn: 0.1704261	total: 5.49s	remaining: 25.4s
178:	learn: 0.1702671	total: 5.53s	remaining: 25.4s
179:	learn: 0.1700242	total: 5.55s	remaining: 25.3s
180:	learn: 0.1697634	total: 5.58s	remaining: 25.2s
181:	learn: 0.1696588	total: 5.6s	remaining: 25.2s
182:	learn: 0.1694864	total: 5.62s	remaining: 25.1s
183:	learn: 0.16920

325:	learn: 0.1470959	total: 9.64s	remaining: 19.9s
326:	learn: 0.1470178	total: 9.67s	remaining: 19.9s
327:	learn: 0.1469336	total: 9.7s	remaining: 19.9s
328:	learn: 0.1468334	total: 9.73s	remaining: 19.8s
329:	learn: 0.1467313	total: 9.75s	remaining: 19.8s
330:	learn: 0.1465580	total: 9.78s	remaining: 19.8s
331:	learn: 0.1465209	total: 9.8s	remaining: 19.7s
332:	learn: 0.1463504	total: 9.83s	remaining: 19.7s
333:	learn: 0.1462301	total: 9.86s	remaining: 19.7s
334:	learn: 0.1461482	total: 9.88s	remaining: 19.6s
335:	learn: 0.1460872	total: 9.91s	remaining: 19.6s
336:	learn: 0.1459676	total: 9.94s	remaining: 19.6s
337:	learn: 0.1457771	total: 9.97s	remaining: 19.5s
338:	learn: 0.1457099	total: 9.99s	remaining: 19.5s
339:	learn: 0.1455554	total: 10s	remaining: 19.5s
340:	learn: 0.1453912	total: 10.1s	remaining: 19.4s
341:	learn: 0.1453759	total: 10.1s	remaining: 19.4s
342:	learn: 0.1452448	total: 10.1s	remaining: 19.4s
343:	learn: 0.1451847	total: 10.1s	remaining: 19.3s
344:	learn: 0.14

486:	learn: 0.1321332	total: 14.2s	remaining: 14.9s
487:	learn: 0.1319781	total: 14.2s	remaining: 14.9s
488:	learn: 0.1319053	total: 14.2s	remaining: 14.9s
489:	learn: 0.1318888	total: 14.3s	remaining: 14.8s
490:	learn: 0.1318021	total: 14.3s	remaining: 14.8s
491:	learn: 0.1316617	total: 14.3s	remaining: 14.8s
492:	learn: 0.1314256	total: 14.3s	remaining: 14.7s
493:	learn: 0.1313908	total: 14.4s	remaining: 14.7s
494:	learn: 0.1313245	total: 14.4s	remaining: 14.7s
495:	learn: 0.1312368	total: 14.4s	remaining: 14.6s
496:	learn: 0.1311718	total: 14.4s	remaining: 14.6s
497:	learn: 0.1310913	total: 14.5s	remaining: 14.6s
498:	learn: 0.1310494	total: 14.5s	remaining: 14.5s
499:	learn: 0.1309711	total: 14.5s	remaining: 14.5s
500:	learn: 0.1308901	total: 14.5s	remaining: 14.5s
501:	learn: 0.1308200	total: 14.6s	remaining: 14.5s
502:	learn: 0.1307333	total: 14.6s	remaining: 14.4s
503:	learn: 0.1306655	total: 14.6s	remaining: 14.4s
504:	learn: 0.1306074	total: 14.7s	remaining: 14.4s
505:	learn: 

647:	learn: 0.1204587	total: 18.7s	remaining: 10.2s
648:	learn: 0.1204074	total: 18.8s	remaining: 10.1s
649:	learn: 0.1203478	total: 18.8s	remaining: 10.1s
650:	learn: 0.1202883	total: 18.8s	remaining: 10.1s
651:	learn: 0.1202365	total: 18.8s	remaining: 10.1s
652:	learn: 0.1201456	total: 18.9s	remaining: 10s
653:	learn: 0.1200797	total: 18.9s	remaining: 10s
654:	learn: 0.1200077	total: 18.9s	remaining: 9.96s
655:	learn: 0.1199745	total: 18.9s	remaining: 9.94s
656:	learn: 0.1199275	total: 19s	remaining: 9.9s
657:	learn: 0.1198921	total: 19s	remaining: 9.87s
658:	learn: 0.1198075	total: 19s	remaining: 9.84s
659:	learn: 0.1197619	total: 19s	remaining: 9.81s
660:	learn: 0.1197038	total: 19.1s	remaining: 9.78s
661:	learn: 0.1196547	total: 19.1s	remaining: 9.76s
662:	learn: 0.1195903	total: 19.1s	remaining: 9.72s
663:	learn: 0.1195295	total: 19.2s	remaining: 9.7s
664:	learn: 0.1194820	total: 19.2s	remaining: 9.66s
665:	learn: 0.1194820	total: 19.2s	remaining: 9.63s
666:	learn: 0.1194698	tota

808:	learn: 0.1117988	total: 23.1s	remaining: 5.46s
809:	learn: 0.1117692	total: 23.2s	remaining: 5.43s
810:	learn: 0.1116983	total: 23.2s	remaining: 5.4s
811:	learn: 0.1116258	total: 23.2s	remaining: 5.37s
812:	learn: 0.1115438	total: 23.2s	remaining: 5.34s
813:	learn: 0.1114636	total: 23.3s	remaining: 5.31s
814:	learn: 0.1113935	total: 23.3s	remaining: 5.29s
815:	learn: 0.1113701	total: 23.3s	remaining: 5.26s
816:	learn: 0.1112693	total: 23.3s	remaining: 5.23s
817:	learn: 0.1112216	total: 23.4s	remaining: 5.2s
818:	learn: 0.1111443	total: 23.4s	remaining: 5.17s
819:	learn: 0.1110456	total: 23.4s	remaining: 5.14s
820:	learn: 0.1110132	total: 23.4s	remaining: 5.11s
821:	learn: 0.1109582	total: 23.5s	remaining: 5.08s
822:	learn: 0.1109384	total: 23.5s	remaining: 5.06s
823:	learn: 0.1109028	total: 23.5s	remaining: 5.03s
824:	learn: 0.1108652	total: 23.6s	remaining: 5s
825:	learn: 0.1108268	total: 23.6s	remaining: 4.97s
826:	learn: 0.1107375	total: 23.6s	remaining: 4.94s
827:	learn: 0.110

968:	learn: 0.1037755	total: 27.7s	remaining: 885ms
969:	learn: 0.1037294	total: 27.7s	remaining: 856ms
970:	learn: 0.1036818	total: 27.7s	remaining: 828ms
971:	learn: 0.1036056	total: 27.7s	remaining: 799ms
972:	learn: 0.1035657	total: 27.8s	remaining: 771ms
973:	learn: 0.1035582	total: 27.8s	remaining: 742ms
974:	learn: 0.1035502	total: 27.8s	remaining: 713ms
975:	learn: 0.1035060	total: 27.8s	remaining: 685ms
976:	learn: 0.1034833	total: 27.9s	remaining: 656ms
977:	learn: 0.1034263	total: 27.9s	remaining: 628ms
978:	learn: 0.1033425	total: 27.9s	remaining: 599ms
979:	learn: 0.1033281	total: 27.9s	remaining: 570ms
980:	learn: 0.1032750	total: 28s	remaining: 542ms
981:	learn: 0.1032144	total: 28s	remaining: 513ms
982:	learn: 0.1031570	total: 28s	remaining: 485ms
983:	learn: 0.1031096	total: 28.1s	remaining: 456ms
984:	learn: 0.1030305	total: 28.1s	remaining: 428ms
985:	learn: 0.1029886	total: 28.1s	remaining: 399ms
986:	learn: 0.1029550	total: 28.1s	remaining: 371ms
987:	learn: 0.1029

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler()),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  array(['additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare'], dtype=object))])),
                ('estimators',
                 <catboost.core.CatBoostClassifier object at 0x00000240555AC1C8>)])

In [54]:
test_features_df = pd.read_csv("test.csv", 
                               index_col="tripid")
submission_df = pd.read_csv("sample_submission.csv",
                            index_col="tripid")

In [55]:
test_probas = full_pipeline.predict(test_features_df)
test_probas

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [56]:
submission_df.head()

,prediction
tripid,
213284604,1
213286352,0
213293973,0
213294622,1
213298687,1


In [57]:
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

submission_df["prediction"] = test_probas

submission_df.to_csv('160180f_catboostWithHours.csv', index=True)

In [58]:
%alias head powershell -command "& {Get-Content %s -Head 10}"
%head my_submission.csv

tripid,prediction
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1
